In [2]:
#Sift Feature Extraction 

%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from utils import *
from img_utils import *

2024-02-28 10:29:33.798742: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
path = "../nybolig-scrape/output"
houses_df_ = data_to_DF(path, max_houses=1000)

[ WARN:0@11.460] global loadsave.cpp:244 findDecoder imread_(''): can't open/read file: check file path/integrity
[ WARN:0@14.941] global loadsave.cpp:244 findDecoder imread_(''): can't open/read file: check file path/integrity


In [58]:
from utils import remove_outliers, prepare_features

#Take only Copenhagen, and Ejerlejligheder
houses_df = houses_df_[(houses_df_['postal_code'] >= 1000) & (houses_df_['postal_code'] <= 2920)]
houses_df = houses_df[houses_df['type'] == 'ejerlejlighed']
houses_df = remove_outliers(houses_df, 'price')
houses_df = houses_df[houses_df['image_floorplan'].notna()]

#Run prepare_features
houses_df = prepare_features(houses_df)
display(houses_df.head(5))



,postal_code,price,size,basement_size,rooms,year_built,year_rebuilt,energy_label,image_floorplan
0,-0.149993,3995000,0.127919,0.0,0.053319,1.295765,1.164627,-1.658253,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
1,1.526638,2325000,-1.012330,0.0,-0.853106,-0.492987,-0.686204,0.809845,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
2,0.488724,2195000,0.453704,0.0,0.959744,0.358799,0.195144,0.192820,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
3,-0.389511,7995000,1.333325,0.0,1.866169,0.252326,0.084976,0.809845,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
4,-1.028228,10200000,1.072697,0.0,0.959744,1.380943,1.252762,-1.658253,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."


In [59]:
#Preprocess the images 
resize = True 
gray_scale = False 
threshhold = True 
img_width, img_height = 500, 500

images = preprocess_images(houses_df, column_name="image_floorplan", width=img_width, height=img_height, resize=resize, gray_scale=gray_scale, threshhold=threshhold)

In [60]:
bow_histograms = create_bow_representation(images)

In [61]:
#Add bow_histograms to the dataframe
houses_df['bow_histograms'] = bow_histograms

In [62]:
from sklearn.model_selection import train_test_split
x = houses_df.drop(['price', 'image_floorplan'], axis=1)
y = houses_df['price']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [63]:
display(X_train.head(5))

,postal_code,size,basement_size,rooms,year_built,year_rebuilt,energy_label,bow_histograms
259,-0.309672,-0.100131,0.0,0.053319,0.443978,0.283279,0.809845,"[0.01509433962264151, 0.0037735849056603774, 0..."
96,-0.309672,-0.491074,0.0,-0.853106,-0.407808,-0.598069,0.192820,"[0.0, 0.01556420233463035, 0.00778210116731517..."
208,2.005675,-0.165288,0.0,0.053319,-0.194862,-0.377732,0.192820,"[0.012232415902140673, 0.0061162079510703364, ..."
130,0.568563,0.290811,0.0,0.053319,1.338354,1.208695,-1.658253,"[0.008670520231213872, 0.011560693641618497, 0..."
128,-0.149993,1.105275,0.0,0.959744,1.295765,1.164627,-1.658253,"[0.00558659217877095, 0.0223463687150838, 0.0,..."


# SVC Using only BoW-Histograms

In [64]:
#Create the bag of words representation
#Run SVM on the bow histograms
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

clf = SVC(kernel='linear')
x_train_bow = [x.flatten() for x in X_train['bow_histograms']]
x_test_bow = [x.flatten() for x in X_test['bow_histograms']]

clf.fit(x_train_bow, y_train)
y_pred = clf.predict(x_test_bow)

#Do Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error: ", mae)

#Do percentage error 
percentage_error = np.mean(np.abs(y_test - y_pred) / y_test)
print("Percentage Error: ", percentage_error)

Mean Absolute Error:  2256073.1707317075
Percentage Error:  0.4312660853276737


# RF Using BoW-Histograms

In [69]:
#Do The same with a Random Forest
from sklearn.ensemble import RandomForestRegressor
Rf = RandomForestRegressor(n_estimators=100)
Rf.fit(x_train_bow, y_train)
y_pred = Rf.predict(x_test_bow)

#Do Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error: ", mae)

#Do percentage error
percentage_error = np.mean(np.abs(y_test - y_pred) / y_test)
print("Percentage Error: ", percentage_error)

[2995950. 3520860. 3666580. 3614040. 5469420. 2923410. 4131150. 4481280.
 5846800. 6779040. 5536480. 3847610. 5695520. 5509730. 5730530. 5232880.
 3446610. 3096430. 4928020. 5170740. 4627620. 4239440. 5908530. 3559820.
 3825750. 5483490. 5047480. 3949830. 4278590. 5402090. 4108540. 3662180.
 4217210. 4286070. 4594520. 4731700. 5070590. 3984880. 2870140. 3177090.
 2990690. 4033260. 3656260. 3335800. 2679740. 4536100. 4716290. 6351980.
 4299290. 3290810. 3380690. 3932360. 5988040. 6621050. 4766940. 4344830.
 4016490. 3656180. 4355070. 3330030. 2653930. 2768710. 4001570. 4609390.
 3863730. 4504170. 5359630. 5275470. 3323030. 3149930. 3331520. 3759610.
 3381200. 3630190. 3335690. 4991360. 4365680. 4194910. 4449260. 4325850.
 3652040. 3544190.]
Mean Absolute Error:  1370012.6829268292
Percentage Error:  0.3546103156300796


# RF Using Bow-Histograms and features

In [70]:
from utils import flatten_columns
drop_columns = ['price', 'image_floorplan']

X_train, X_test, y_train, y_test = train_test_split(houses_df.drop(drop_columns, axis=1), 
                                                    houses_df['price'], test_size=0.3, random_state=42)


X_train = flatten_columns(X_train, 'bow_histograms')
X_test = flatten_columns(X_test, 'bow_histograms')
display(X_train.head(5))

,postal_code,size,basement_size,rooms,year_built,year_rebuilt,energy_label,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
259,-0.309672,-0.100131,0.0,0.053319,0.443978,0.283279,0.809845,0.015094,0.003774,0.033962,0.000000,0.018868,0.015094,0.022642,0.026415,0.000000,0.007547,0.037736,0.018868,0.030189,0.018868,0.011321,0.000000,0.003774,0.015094,0.003774,0.003774,0.003774,0.003774,0.000000,0.003774,0.003774,0.003774,0.007547,0.015094,0.007547,0.003774,0.007547,0.015094,0.007547,...,0.011321,0.007547,0.003774,0.011321,0.018868,0.000000,0.015094,0.000000,0.003774,0.003774,0.003774,0.003774,0.003774,0.022642,0.000000,0.011321,0.022642,0.030189,0.003774,0.007547,0.007547,0.011321,0.011321,0.011321,0.015094,0.007547,0.022642,0.007547,0.015094,0.018868,0.007547,0.011321,0.007547,0.000000,0.003774,0.018868,0.007547,0.000000,0.007547,0.007547
96,-0.309672,-0.491074,0.0,-0.853106,-0.407808,-0.598069,0.192820,0.000000,0.015564,0.007782,0.011673,0.011673,0.007782,0.035019,0.011673,0.003891,0.007782,0.031128,0.023346,0.019455,0.003891,0.023346,0.011673,0.007782,0.015564,0.019455,0.011673,0.003891,0.003891,0.023346,0.000000,0.023346,0.011673,0.011673,0.003891,0.011673,0.007782,0.000000,0.003891,0.007782,...,0.003891,0.007782,0.019455,0.007782,0.007782,0.019455,0.011673,0.003891,0.019455,0.015564,0.015564,0.003891,0.003891,0.003891,0.007782,0.011673,0.000000,0.003891,0.011673,0.003891,0.015564,0.003891,0.019455,0.007782,0.011673,0.011673,0.015564,0.007782,0.011673,0.015564,0.003891,0.007782,0.000000,0.011673,0.011673,0.011673,0.007782,0.007782,0.007782,0.003891
208,2.005675,-0.165288,0.0,0.053319,-0.194862,-0.377732,0.192820,0.012232,0.006116,0.006116,0.003058,0.006116,0.006116,0.018349,0.027523,0.000000,0.015291,0.021407,0.018349,0.030581,0.000000,0.006116,0.018349,0.009174,0.000000,0.036697,0.012232,0.003058,0.000000,0.006116,0.018349,0.000000,0.000000,0.009174,0.003058,0.021407,0.003058,0.003058,0.009174,0.030581,...,0.009174,0.003058,0.000000,0.015291,0.003058,0.009174,0.000000,0.024465,0.018349,0.003058,0.039755,0.006116,0.003058,0.015291,0.012232,0.024465,0.012232,0.003058,0.009174,0.006116,0.015291,0.003058,0.009174,0.009174,0.018349,0.009174,0.015291,0.012232,0.021407,0.021407,0.006116,0.024465,0.015291,0.006116,0.003058,0.006116,0.003058,0.006116,0.003058,0.000000
130,0.568563,0.290811,0.0,0.053319,1.338354,1.208695,-1.658253,0.008671,0.011561,0.043353,0.008671,0.011561,0.008671,0.031792,0.026012,0.005780,0.008671,0.020231,0.031792,0.023121,0.005780,0.002890,0.014451,0.002890,0.005780,0.008671,0.008671,0.008671,0.008671,0.014451,0.000000,0.011561,0.026012,0.023121,0.008671,0.008671,0.005780,0.005780,0.005780,0.005780,...,0.002890,0.020231,0.008671,0.008671,0.011561,0.011561,0.011561,0.008671,0.002890,0.002890,0.008671,0.005780,0.002890,0.005780,0.008671,0.002890,0.005780,0.005780,0.002890,0.008671,0.023121,0.008671,0.020231,0.008671,0.000000,0.011561,0.005780,0.011561,0.014451,0.008671,0.005780,0.005780,0.008671,0.002890,0.002890,0.008671,0.005780,0.002890,0.008671,0.011561
128,-0.149993,1.105275,0.0,0.959744,1.295765,1.164627,-1.658253,0.005587,0.022346,0.000000,0.000000,0.016760,0.000000,0.011173,0.050279,0.000000,0.000000,0.005587,0.039106,0.050279,0.005587,0.033520,0.011173,0.016760,0.000000,0.000000,0.000000,0.000000,0.005587,0.005587,0.005587,0.000000,0.005587,0.044693,0.000000,0.011173,0.000000,0.005587,0.005587,0.011173,...,0.000000,0.000000,0.000000,0.005587,0.000000,0.011173,0.000000,0.000000,0.005587,0.000000,0.005587,0.033520,0.000000,0.055866,0.000000,0.027933,0.005587,0.000000,0.016760,0.005587,0.033520,0.011173,0.016760,0.016760,0.000000,0.016760,0.000000,0.016760,0.016760,0.039106,0.000000,0.061453,0.011173,0.005587,0.000000,0.005587,0.000000,0.000000,0.000000,0.005587


In [71]:
#Do Random Forest on the features
Rf = RandomForestRegressor(n_estimators=1000)
Rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1000)

In [73]:
#Do Mean Absolute Error
y_pred = Rf.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error: ", mae)

#Do percentage error
percentage_error = np.mean(np.abs(y_test - y_pred) / y_test)
print("Percentage Error: ", percentage_error)

Mean Absolute Error:  858964.256097561
Percentage Error:  0.21589324403683233
